# PS7

MACS 30100 Dr. Evens

Jie Heng

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import pandas as pd
import numpy as np

auto = pd.read_csv('Auto.csv', na_values='?')
auto = auto.dropna()
auto.isna().sum()

mpg             0
cylinders       0
displacement    0
horsepower      0
weight          0
acceleration    0
year            0
origin          0
name            0
dtype: int64

## Problem 1 Classiﬁer “horse” race 

### (a)   Use sklearn.linear model.LogisticRegression to ﬁt a logistic model of mpg high on features number of cylinders (cyl), engine displacement (dspl), horsepower (hpwr), vehicle weight (wgt), acceleration (accl), vehicle year (yr), vehicle origin (orgn). Make sure to include a constant term. Fit the model using k-fold cross validation with k = 4 folds. Report the MSE of the model as the average MSE across the k = 4 test sets, and report the error rates for each category of mpg high as the average error rate for that category across the k = 4 test sets.

In [2]:
auto = auto.drop(columns = ["name"])
cutoff = auto["mpg"].median()

def cut(col):
    mpg = col
    if mpg >= cutoff:
        return 1
    else:
        return 0
    
auto["mpg_high"] = auto["mpg"].apply(cut)
auto.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,mpg_high
0,18.0,8,307.0,130.0,3504,12.0,70,1,0
1,15.0,8,350.0,165.0,3693,11.5,70,1,0
2,18.0,8,318.0,150.0,3436,11.0,70,1,0
3,16.0,8,304.0,150.0,3433,12.0,70,1,0
4,17.0,8,302.0,140.0,3449,10.5,70,1,0


In [3]:
Xvars = auto[['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year', 'origin']].values
yvals = auto['mpg_high'].values

In [4]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)
Log_error_0 = np.zeros(k)
Log_error_1 = np.zeros(k)
Log_MSE = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(Xvars):

    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    error = y_test != y_pred
    error_all = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum()
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum() 
    Log_error_0[k_ind] = error_0
    Log_error_1[k_ind] = error_1
    Log_MSE[k_ind] = error_all
    
    print('When k index=', k_ind, ':')
    print('')
    print(classification_report(y_test, y_pred, digits=3))
    print('')
    print('Error rate for category 0 is', error_0)
    print('Error rate for category 1 is', error_1)      
    print('The MSE of the model is', error_all)
    print('')
    k_ind += 1

When k index= 0 :

             precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98


Error rate for category 0 is 0.057692307692307696
Error rate for category 1 is 0.13043478260869565
The MSE of the model is 0.09183673469387756

When k index= 1 :

             precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98


Error rate for category 0 is 0.12244897959183673
Error rate for category 1 is 0.08163265306122448
The MSE of the model is 0.10204081632653061

When k index= 2 :

             precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0.867     0.867        98


Error ra

In [5]:
print("The average MSE across the k = 4 test sets is", Log_MSE.mean())
print("The average error rate for category 0 is", Log_error_0 .mean())
print("The average error rate for category 1 is", Log_error_1.mean())

The average MSE across the k = 4 test sets is 0.10714285714285715
The average error rate for category 0 is 0.09470670705864986
The average error rate for category 1 is 0.11822664912727025


### (b) Use sklearn.ensemble.RandomForestClassifier to ﬁt a random forest model of mpg high on max features=2 out of the seven possible features used in part (a). Set n estimators=20, set bootstrap=True, set oob score=True, and set random state=25. Report the MSE of the random forest model as the MSE from the .oob prediction object, and report the error rates for each category of mpg high from the .oob prediction object.

In [6]:
RF = RandomForestClassifier(n_estimators=20, max_features=2, bootstrap=True,oob_score=True, random_state=25)
RF.fit(Xvars, yvals)
oob_prediction = RF.oob_decision_function_.T[1]
MSE_RF = pd.DataFrame({'pred' : oob_prediction, 'yvals': yvals})
MSE_RF['pred'] = MSE_RF['pred'].apply(lambda x: 1 if x >= 0.5 else 0)
print(classification_report(MSE_RF['yvals'], MSE_RF['pred']))
error = (MSE_RF['pred'] != MSE_RF['yvals']).apply(int)
RF_err_0 = ((MSE_RF['pred'] == 0) * error).sum() / (MSE_RF['pred'] == 0).sum() 
RF_err_1 = ((MSE_RF['pred'] == 1) * error).sum() / (MSE_RF['pred'] == 1).sum() 

             precision    recall  f1-score   support

          0       0.94      0.91      0.93       196
          1       0.92      0.94      0.93       196

avg / total       0.93      0.93      0.93       392



In [7]:
print('The MSE of the random forest model ', error.mean())
print('The error rate for category 0 is', RF_err_0.mean())
print('The error rate for category 1 is', RF_err_1.mean())

The MSE of the random forest model  0.07142857142857142
The error rate for category 0 is 0.05789473684210526
The error rate for category 1 is 0.08415841584158416


### (c) Use sklearn.svm.SVC to ﬁt a support vector machines model of mpg high with a Gaussian radial basis function kernel kernel=’rbf’ on the seven features used in part (a). Set the penalty parameter to C=1 and set gamma=0.2. Fit the model using k-fold cross validation with k = 4 folds exactly as in part (a). Report the MSE of the model as the average MSE across the k = 4 test sets, and report the error rates for each category of mpg high as the average error rate for that category across the k = 4 test sets. 

In [8]:
C = 1
gamma = 0.2

In [9]:
k = 4
kf = KFold(n_splits=k, random_state=15, shuffle=True)
kf.get_n_splits(Xvars)
SVMGeneral_error_0 = np.zeros(k)
SVMGeneral_error_1 = np.zeros(k)
SVMGeneral_MSE = np.zeros(k)
k_ind = int(0)

for train_index, test_index in kf.split(Xvars):
    X_train, X_test = Xvars[train_index], Xvars[test_index]
    y_train, y_test = yvals[train_index], yvals[test_index]
    svc = svm.SVC(kernel='rbf', gamma=0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred
    error_all = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum()
    SVMGeneral_error_0[k_ind] = error_0
    SVMGeneral_error_1[k_ind] = error_1
    SVMGeneral_MSE[k_ind] = error_all
    print('When k index=', k_ind,':')
    print('')
    print(classification_report(y_test, y_pred, digits=3))
    print('')
    print('Error rate for category 0 is', error_0)
    print('Error rate for category 1 is', error_1)      
    print('The MSE of the model is', error_all)
    print('')
    k_ind += 1

When k index= 0 :

             precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98


Error rate for category 0 is 0.0
Error rate for category 1 is 0.5520833333333334
The MSE of the model is 0.5408163265306123

When k index= 1 :

             precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98


Error rate for category 0 is 0.5204081632653061
Error rate for category 1 is nan
The MSE of the model is 0.5204081632653061

When k index= 2 :

             precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98


Error rate for category 0 is 0.53125
Error ra

C:\Users\hengj\Anaconda_3\lib\site-packages\ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in long_scalars
C:\Users\hengj\Anaconda_3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
SVMGeneral_error_1 = SVMGeneral_error_1[~np.isnan(SVMGeneral_error_1)]
print("The average MSE across the k = 4 test sets is", SVMGeneral_MSE.mean())
print("The average error rate for category 0 is", SVMGeneral_error_0.mean())
print("The average error rate for category 1 is", SVMGeneral_error_1.mean())

The average MSE across the k = 4 test sets is 0.5076530612244898
The average error rate for category 0 is 0.38119411070879966
The average error rate for category 1 is 0.1840277777777778


### (d) Which of the above three models do you think is the best predictor of mpg high? Why?

The Average MSE for logistic model is 0.1071, the average MSE for random forest model is 0.0714, and the average MSE for svc model is 0.5077. We can see that the randome forest model has the least amount of average MSE. For the error rates for each category of mpg high, the random forest model also has the smallest error rates. Thus, the random forest model is the best predictor of mpg_high.